https://oikolab.com/documentation

In [1]:
import pandas as pd
import json
import requests
import time
import datetime

In [2]:
import json
import requests
import time

# states from usa_merged_data
usa_states = ['Wyoming', 'Wisconsin', 'West Virginia', 'Washington', 'Virginia',
              'Vermont', 'Utah', 'Texas', 'Tennessee', 'South Dakota',
              'South Carolina', 'Rhode Island', 'Pennsylvania', 'Oregon',
              'Oklahoma', 'Ohio', 'North Dakota', 'North Carolina', 'New York',
              'New Mexico', 'New Jersey', 'New Hampshire', 'Nevada', 'Nebraska',
              'Montana', 'Missouri', 'Mississippi', 'Minnesota', 'Michigan',
              'Massachusetts', 'Maryland', 'Maine', 'Louisiana', 'Kentucky',
              'Kansas', 'Iowa', 'Indiana', 'Illinois', 'Idaho', 'Hawaii',
              'Georgia', 'Florida', 'District of Columbia', 'Delaware',
              'Connecticut', 'Colorado', 'California', 'Arkansas', 'Arizona',
              'Alaska', 'Alabama']

weather_usa_states = pd.DataFrame()

today = datetime.datetime.today()
yesterday = (today - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

i=1

for state in usa_states:
    
    r = requests.get('https://api.oikolab.com/weather',
                     params={'param': ['temperature', 'wind_speed'], 
                             'start': '2020-01-01T00:00:00',
                             'end': yesterday+'T00:00:00',
                             'location': state,
                             'api-key': 'xxxxxxxxxxxxxxxxxxxxxx'}
                    )

    if r.status_code == 200:
        attributes = r.json()['attributes']
        weather_data = json.loads(r.json()['data'])

        df = pd.DataFrame(index = pd.to_datetime(weather_data['index'], unit='s'),
                          data = weather_data['data'],
                          columns = weather_data['columns']) \
               .reset_index() \
               .rename(columns={'index': 'date'})

        df['date'] = df['date'].dt.floor('d')

        weather_by_day = df.groupby('date').median().reset_index()
        weather_by_day['state'] = state

        weather_usa_states = pd.concat([weather_by_day, weather_usa_states])


    else:
        print(r.text)
        
    i +=1
    
    if ((i%5 == 0) & (i<50)):
        time.sleep(61)

{ "statusCode": 429, "message": "Rate limit is exceeded. Try again in 45 seconds." }
{ "statusCode": 429, "message": "Rate limit is exceeded. Try again in 44 seconds." }


In [4]:
weather_usa_states.head()

,date,temperature (degC),wind_speed (m/s),state
0,2020-01-01,3.72,1.350,Arizona
1,2020-01-02,1.11,2.005,Arizona
2,2020-01-03,1.61,2.135,Arizona
3,2020-01-04,2.23,1.685,Arizona
4,2020-01-05,3.58,1.520,Arizona


In [5]:
weather_usa_states['state'].nunique()

49

In [6]:
len(usa_states)

51

It's missing two states. It probably happended because my limitation plan : "Subscribers will be able to run 5 calls/minute up to a maximum of **300 calls/month**."

In [8]:
weather_usa_states.to_csv('weather_usa_states.csv')